# Analyzing Global Sea Level in GFDL Ocean Models




In [ ]:
! pip  install momlevel intake intake-esm

In [ ]:
import xarray as xr
xr.set_options(display_style='html')
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import momlevel
import fsspec
import gcsfs
import intake
import nc_time_axis

%matplotlib inline

In [ ]:
cat_url = "https://storage.googleapis.com/cmip6/pangeo-cmip6.json"
col = intake.open_esm_datastore(cat_url)
col

In [ ]:
datasets = col.search(
    experiment_id=['historical'],
    table_id=['Ofx','Omon'],
    variable_id=['areacello','zos'],
    grid_label='gn',
    member_id='r1i1p1f1',
    source_id="GFDL-ESM4"
)

df = datasets.df

In [ ]:
df

In [ ]:
zstores = df.zstore.values
print(zstores)

fs = gcsfs.GCSFileSystem(token='anon', access='read_only')

# create a MutableMapping from a store URL
mappers = [fs.get_mapper(x) for x in zstores]

# make sure to specify that metadata is consolidated
ds = [xr.open_zarr(x, consolidated=True) for x in mappers]

ds = xr.merge(ds)

In [ ]:
fig = plt.figure(figsize=(8,3), dpi=150)

ax1 = plt.subplot(1,2,1)
ds.areacello.plot(ax=ax1)
_ = ax1.set_title("Cell Area")

ax2 = plt.subplot(1,2,2)
ds.zos.isel(time=-1).plot(ax=ax2)
_ = ax2.set_title("Dynamic Sea Surface Height")

plt.subplots_adjust(wspace=0.5)

In [ ]:
ssh_global_monthly = ds.zos.weighted(ds.areacello).mean(("y","x")).load()
ssh_global_annual = momlevel.util.annual_average(ssh_global_monthly).load()

In [ ]:
fig = plt.figure(figsize=(8,4), dpi=150)
ax = plt.subplot(1,1,1)

ax.plot(ssh_global_monthly.time.values,ssh_global_monthly.values, linewidth=0.5, color="lightgray")
ax.plot(ssh_global_annual.time.values,ssh_global_annual.values, linewidth=1.2, color="red")

ax.set_title("Global Mean Dynamic Sea Surface Height")
ax.set_ylabel("m")

In [ ]:
datasets = col.search(
    experiment_id=['historical'],
    table_id=['Ofx','Omon'],
    variable_id=['volcello','thetao', 'so', 'areacello'],
    grid_label='gn',
    member_id='r1i1p1f1',
    source_id="GFDL-ESM4"
)

df = datasets.df


In [ ]:
df

In [ ]:
zstores = df.zstore.values
print(zstores)

fs = gcsfs.GCSFileSystem(token='anon', access='read_only')

# create a MutableMapping from a store URL
mappers = [fs.get_mapper(x) for x in zstores]

# make sure to specify that metadata is consolidated
ds = [xr.open_zarr(x, consolidated=True) for x in mappers]

ds = xr.merge(ds)

In [ ]:
ds["deptho"] = xr.open_zarr("https://extranet.gfdl.noaa.gov/~John.Krasting/models/GFDL-ESM4/deptho").deptho.load()

In [ ]:
ds

In [ ]:
steric = momlevel.steric(ds.isel(time=slice(-360,None)), domain="global", coord_names={"z":"lev"})

In [ ]:
steric_change = steric[0].steric.load()